<a href="https://colab.research.google.com/github/SovetovAleksey/Data_analysis_algorithms/blob/4_quest/4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import r2_score

In [2]:
# сгенерируем набор данных
data, target, coef = datasets.make_regression(n_samples=1000, n_features = 3, n_informative = 2, n_targets = 1,
                                              noise = 5, coef = True, random_state = 2)

In [3]:
def data_std(data):
    return data - data.mean() / data.std()

In [4]:
data = data_std(data)

In [5]:
# Реализуем класс узла
class Node:

    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [6]:
# И класс терминального узла (листа)
class Leaf:

    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.prediction = self.predict()

    def predict(self):
        return self.target.mean()

In [7]:
# Расчет дисперсии
def variance(target):
    return np.sum(((target - target.mean()) ** 2)) / target.size

In [8]:
# Расчет качества
def quality(left_target, right_target, current_variance):
    # доля выбоки, ушедшая в левое поддерево
    p = float(left_target.shape[0]) / (left_target.shape[0] + right_target.shape[0])

    return current_variance - p * variance(left_target) - (1 - p) * variance(right_target)

In [9]:
# Разбиение датасета в узле
def split(data, target, index, t):
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_target = target[left]
    false_target = target[right]

    return true_data, false_data, true_target, false_target

In [10]:
# Нахождение наилучшего разбиения

def find_best_split(data, target, min_leaf=5):

    current_variance = variance(target)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_target, false_target = split(data, target, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_target, false_target, current_variance)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [11]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, target, min_leaf=5, level=-1):

    quality, t, index = find_best_split(data, target, min_leaf=min_leaf)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, target)


    true_data, false_data, true_target, false_target = split(data, target, index, t)

    # Рекурсивно строим два поддерева, пока не будет достигнуто нужное число уровней, если оно указано при вызове
    if level < 0 or level >= 1: 
        true_branch = build_tree(true_data, true_target, min_leaf=min_leaf, level=level-1)
        false_branch = build_tree(false_data, false_target, min_leaf=min_leaf, level=level-1)
    else:
        return Leaf(data, target)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [12]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [13]:
def predict(data, tree):
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [14]:
# Разобьем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data,
                                                                    target,
                                                                    test_size = 0.3,
                                                                    random_state = 1)

In [15]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_target, min_leaf=10, level=5)

In [16]:
# Получим ответы для обучающей выборки
train_answers = predict(train_data, my_tree)

# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

r2_train = r2_score(list(train_target), train_answers)
r2_test = r2_score(list(test_target), answers)
print(r2_train, r2_test)

0.9605874379857027 0.9250920385959558


In [17]:
class ColorText:
    PURPLE = '\033[1;35;48m'
    CYAN = '\033[1;36;48m'
    BOLD = '\033[1;39;48m'
    GREEN = '\033[1;34;48m'
    BLUE = '\033[1;44;48m'
    ORANGE = '\033[1;32;48m'
    YELLOW = '\033[1;33;48m'
    RED = '\033[1;31;48m'
    BLACK = '\033[1;30;48m'
    UNDERLINE = '\033[1;37;48m'
    END = '\033[1;37;0m'

In [18]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(ColorText.ORANGE + spacing + ' ЛИСТ' 
                  + ': прогноз = ' + str(node.prediction) 
                  + ', объектов = ' + str(len(node.target)) 
                  + ColorText.END)
        return

    # Выведем значение индекса и порога на этом узле
    print(ColorText.GREEN + spacing + 'УЗЕЛ'  
              + ': индекс = ' + str(node.index) 
              + ', порог = ' + str(round(node.t, 2))
              + ColorText.END)

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> Левая ветка:')
    print_tree(node.true_branch, spacing + "   ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> Правая ветка:')
    print_tree(node.false_branch, spacing + "   ")
    
print_tree(my_tree)

УЗЕЛ: индекс = 1, порог = -0.09
--> Левая ветка:
   УЗЕЛ: индекс = 1, порог = -1.01
   --> Левая ветка:
      УЗЕЛ: индекс = 2, порог = -0.53
      --> Левая ветка:
         УЗЕЛ: индекс = 1, порог = -1.84
         --> Левая ветка:
             ЛИСТ: прогноз = -208.2622132929394, объектов = 10
         --> Правая ветка:
            УЗЕЛ: индекс = 2, порог = -0.86
            --> Левая ветка:
                ЛИСТ: прогноз = -156.7966962492283, объектов = 20
            --> Правая ветка:
                ЛИСТ: прогноз = -126.45251969376058, объектов = 13
      --> Правая ветка:
         УЗЕЛ: индекс = 1, порог = -1.73
         --> Левая ветка:
            УЗЕЛ: индекс = 2, порог = 0.34
            --> Левая ветка:
                ЛИСТ: прогноз = -143.77597766027958, объектов = 10
            --> Правая ветка:
                ЛИСТ: прогноз = -107.36446810020804, объектов = 10
         --> Правая ветка:
            УЗЕЛ: индекс = 2, порог = 0.73
            --> Левая ветка:
                